### We willbe implemtning the word2_Vecsuing varius pretrianed modelsin the library.

### 1. Implementing Word_2_vec.

In [1]:
import gensim
import gensim.downloader as api

In [2]:
gensim.downloader.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

### Loading the Data.

In [6]:
word_vectors=api.load('glove-wiki-gigaword-50')

### Checking the similaritites

In [9]:
word_vectors.most_similar('this')

[('it', 0.9442822337150574),
 ('same', 0.9386158585548401),
 ('.', 0.9227082133293152),
 ('fact', 0.9213975071907043),
 ('that', 0.9183594584465027),
 ('but', 0.9125312566757202),
 ('though', 0.9103720784187317),
 ('yet', 0.9091917276382446),
 ('however', 0.9052569270133972),
 ('as', 0.9036165475845337)]

In [10]:
vec = word_vectors['king'] - word_vectors['man'] + word_vectors['women']
word_vectors.most_similar(vec)
     

[('kingdom', 0.7013998031616211),
 ('king', 0.6653268933296204),
 ('women', 0.6566213369369507),
 ('queen', 0.629289984703064),
 ('invited', 0.6251225471496582),
 ('ii', 0.6004623174667358),
 ('ceremonies', 0.5975189208984375),
 ('commonwealth', 0.5949116349220276),
 ('participate', 0.5945374369621277),
 ('attend', 0.5938369035720825)]

In [11]:
vec = word_vectors['paris'] - word_vectors['france'] + word_vectors['italy']
word_vectors.most_similar(vec)

[('rome', 0.837868332862854),
 ('italy', 0.7875028848648071),
 ('milan', 0.7715330719947815),
 ('turin', 0.7602784633636475),
 ('venice', 0.7574506402015686),
 ('paris', 0.7518839240074158),
 ('madrid', 0.745832622051239),
 ('aires', 0.7402974367141724),
 ('naples', 0.7388359904289246),
 ('italian', 0.737312376499176)]

### Working with the Average Word_2_VEc (for whole sentences).

In [12]:
import nltk
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deeps\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import zipfile


zip_file_path = 'Fake-News-Classification/Fake.zip'
extract_to_folder = 'Fake-News-Classification'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)

print("Extraction completed.")

zip_file_path = 'Fake-News-Classification/True.zip'
extract_to_folder = 'Fake-News-Classification'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)

print("Extraction completed.")


Extraction completed.
Extraction completed.


In [20]:
df = pd.read_csv('Fake-News-Classification/Fake.csv')
df = df.drop(columns = ['date','subject','text'])
df.head()

,title
0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...


In [21]:
words = []
corpus = df['title']

for sent in corpus:
  sent_token = sent_tokenize(sent)
  for sent in sent_token:
      # simple preprocess from gensim is like the word tokenize
    words.append(simple_preprocess(sent))

In [22]:
model = gensim.models.Word2Vec(words, window = 5, min_count = 1)

In [25]:
model.wv['donald'].shape()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

### Average of the vectros in the sentences  <br>
it may occur or not in the model trianed on th e words so it will work accordingly.

In [29]:

sent_vec = []

for sent in words:
    sent_vec.append([model.wv['word'] for word in sent])

In [33]:
sent_vec[:5][0]

[('donald',
  array([-0.02224904,  0.31021136,  0.0178804 ,  0.05863945, -0.02175483,
         -0.65095246,  0.32893226,  0.5625338 , -0.39041826, -0.37974235,
         -0.09191049, -0.386788  ,  0.16851847,  0.21314006,  0.12604748,
         -0.09754136,  0.34646308, -0.40830553,  0.06141304, -0.8220252 ,
          0.20919997, -0.03976647,  0.20897761, -0.3523758 , -0.06249995,
         -0.02802281, -0.21527144, -0.23912837, -0.18265101, -0.13225743,
          0.4429749 ,  0.08078134,  0.04650141, -0.38212916,  0.03839877,
          0.4488523 ,  0.08994576,  0.09268972, -0.23462354, -0.38122243,
          0.18604676, -0.22939084, -0.19941354,  0.3188512 ,  0.4141657 ,
         -0.02579138, -0.39467296,  0.04656895,  0.14972927,  0.28815055,
          0.07610239, -0.20536973, -0.10185207,  0.07101586,  0.07752207,
          0.11675044,  0.30379057,  0.21368562, -0.13375944,  0.1039321 ,
          0.16241345,  0.04568237,  0.05983807,  0.06918543, -0.3793427 ,
          0.32738125, -0.0

In [36]:
sent_vec = []
for sent in tqdm(words):
   sent_vec.append(np.mean([model.wv[word] for word in sent if word in model.wv.index_to_key], axis = 0))
    
sent_vec=np.array(sent_vec)


100%|██████████████████████████████████████████████████████████████████████████| 27474/27474 [00:19<00:00, 1403.82it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (27474,) + inhomogeneous part.

### Working with the Spacy.

In Spacy we could work with the Pipelines accessed throught the official site of spacy.

In [37]:
import spacy
import numpy as np
import pandas as pd

!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     -------------------------------------- 0.0/400.7 MB 325.1 kB/s eta 0:20:33
     -------------------------------------- 0.1/400.7 MB 726.2 kB/s eta 0:09:12
     ---------------------------------------- 0.9/400.7 MB 4.7 MB/s eta 0:01:25
     --------------------------------------- 2.9/400.7 MB 12.2 MB/s eta 0:00:33
     --------------------------------------- 4.9/400.7 MB 17.3 MB/s eta 0:00:23
      -------------------------------------- 6.3/400.7 MB 19.3 MB/s eta 0:00:21
      -------------------------------------- 7.8/400.7 MB 21.8 MB/s eta 0:00:19
      -------------------------------------- 9.9/400.7 MB 24.2 MB/s eta 0:00:17
      ------------------------------------- 10.5/400.7 MB 38.6 MB/s eta 0:00:11
      ------------------------------------- 10.5/400.7 MB 38.6 MB/s eta 0:00:11
      ------------------------------------- 10.5/400.7

In [38]:
nlp = spacy.load("en_core_web_lg")

#### Exploring the Word Vectors.

In [40]:
print('Vector Size : ', nlp.vocab.vectors['car'].shape)
print('Word ID     : ', nlp.vocab.strings['car'])
     

Vector Size :  (300,)
Word ID     :  17545852598994811774


#### Finding the Most Similar Words.

In [41]:
most_similar = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings['delhi']]]), n = 10)

for word in most_similar[0][0]:
  print(nlp.vocab.strings[word])
     

DELHI
MUMBAI
BANGALORE
chennai
PUNE
HYDERABAD
kolkata
INDIA
CHANDIGARH
JAIPUR


#### Dealing with the Sentences | Sentence Embedding 

In [43]:
doc1 = nlp('I like cricket')
doc2 = nlp('Mars in red')
doc3=nlp('I Hate Volleyball')
print(doc1.similarity(doc2))
print(doc3.similarity(doc1))

0.4563753851123069
0.7900762280851414


#### OOV or Out of Vocab

In [45]:
tokens = nlp('Woow, The weather is great')

pd.DataFrame([(token.text, token.has_vector, token.is_oov) for token in tokens], columns = ['Token','Vector','OOV'])
     

,Token,Vector,OOV
0,Woow,True,False
1,",",True,False
2,The,True,False
3,weather,True,False
4,is,True,False
5,great,True,False
